In [2]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
import pandas_gbq as pbq
import numpy as np
from noaa.ghcnd import NOAAStore

In [3]:
store = NOAAStore()

In [4]:
inv = store.stations_inventory(element='PRCP', include_info=True)
inv_active = inv.loc[lambda df:(df['lastyear']>=2022)].copy()
inv_active_1950 = inv_active.loc[lambda df:(df['firstyear']<1950)].groupby('country')['country'].count().sort_values(ascending=False)
inv_active_1950.head(10)

country
US    3472
AS    1782
RS     477
GM     248
SW     178
IN     102
CA      99
NO      78
SF      39
SP      37
Name: country, dtype: int64

In [5]:
prcp = store.timeseries_universe(element='PRCP')

In [5]:
count_1950 = prcp.loc['1950-01-01':].groupby('id').count()
count_1950_90pct = count_1950[(count_1950/count_1950.max()>=0.9)]

In [6]:
prcp_filtered = prcp.loc[slice(None), count_1950_90pct.index.values].copy()
# prcp_filtered_2 = prcp_filtered.copy().reorder_levels(['id','date']).sort_index()

In [50]:
def coverage(s):
    start = s.index.get_level_values('date').min()
    end = s.index.get_level_values('date').max()
    count = s.count()
    length = (end-start).days+1
    coverage = count/length
    return {'start':start,'end':end,'coverage':coverage}
    

In [77]:
prcp_stats = pd.DataFrame(prcp.groupby('id').apply(coverage)).unstack(1)['value'].sort_values('coverage')
prcp_stats

In [84]:
prcp_stats.loc[lambda df:(df['start']<='1950-01-01')&(df['end']>='2022-01-01')&(df['coverage']>=0.90)].sort_values('start')

,start,end,coverage
id,,,
USC00451504,1900-01-01,2022-12-14,0.900597
USC00114108,1900-01-01,2022-12-14,0.963013
USC00226177,1900-01-01,2022-12-18,0.963484
USC00113335,1900-01-01,2022-09-30,0.963665
USW00014946,1900-01-01,2022-12-17,0.963773
...,...,...,...
USC00122731,1949-11-06,2022-12-18,0.90182
USC00046675,1949-12-01,2022-10-31,0.954493
USC00336405,1949-12-01,2022-12-12,0.994227
